観測モデルver9.5(10?)の作成，プリズム制約の勘違い解消版

In [3]:
# 何回目すか,,マジで
import pandas as pd 
import os
import datetime
import numpy as np
from datetime import timedelta 
import csv
import time
import math
from scipy.stats import norm
from scipy.optimize import minimize
import random


In [4]:
df_data = pd.read_csv('/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_0845_20sec.csv')
search_folder = "/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/split_files/group_3_1/group_2"
file_list = list(os.listdir(search_folder))

df_link = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_link_post_former.csv')
L = len(df_link)

df_node = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_node.csv')
df_ble = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_nw_detail/ble_nw.csv')

In [6]:
###### 各linkのoとdの座標を入れておく配列 ###### 
link_loc_array = [] 
for i in range(len(df_link)):
    linkid = df_link.loc[i, 'linkid']
    O = df_link.loc[i, 'o']
    D = df_link.loc[i, 'd']

    x_o = df_node[df_node['nodeid'] == O]['x'].iloc[0]
    y_o = df_node[df_node['nodeid'] == O]['y'].iloc[0]
    z_o = df_node[df_node['nodeid'] == O]['floor'].iloc[0] 
    o_loc = np.array([x_o, y_o, z_o])

    x_d = df_node[df_node['nodeid'] == D]['x'].iloc[0]
    y_d = df_node[df_node['nodeid'] == D]['y'].iloc[0]
    z_d = df_node[df_node['nodeid'] == D]['floor'].iloc[0]
    d_loc = np.array([x_d, y_d, z_d])

    loc_tuple = (o_loc, d_loc)

    link_loc_array.append(loc_tuple) # link_loc_array[linkid//2][0]がlinkidのonode, [1]がdnode，[2]が中点，，，，

###### 各ビーコンの座標を入れておく配列 ###### 
beacon_loc_array = []
for j in range(len(df_ble)):
    beacon_id = df_ble.loc[j, 'ID']
    x_j = df_ble[df_ble['ID'] == beacon_id]['x'].iloc[0]
    y_j = df_ble[df_ble['ID'] == beacon_id]['y'].iloc[0]
    z_j = df_ble[df_ble['ID'] == beacon_id]['floor'].iloc[0]
    beacon_loc = np.array([x_j, y_j, z_j])

    beacon_loc_array.append(beacon_loc) # beacon_loc_array[i]がビーコンi+1の位置座標

###### ビーコンとリンク線分の中点間距離を返す関数 ######
def dist_to_mid(p1, p2, x):
    p1 = np.array(p1)
    p2 = np.array(p2)
    x = np.array(x)
    
    segment = p2 - p1
    v1 = x - p1
    v2 = x - p2
    mid_point = (p1 + p2) / 2

    # 垂直ベクトルを計算
    v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
    distance = np.linalg.norm(x - mid_point)
    
    return distance

###### d_arrayの用意 ###### # 各リンク(の中点）とbleビーコンとの距離配列
d_array = np.zeros((len(df_link), len(df_ble)))

for i in range(len(df_link)):
    p_o = link_loc_array[i][0] # o座標 # i=1の時
    p_d = link_loc_array[i][1] # d座標

    for j in range(len(df_ble)):
        x_ap = df_ble.loc[j, 'x']
        y_ap = df_ble.loc[j, 'y']
        z_ap = df_ble.loc[j, 'floor']
        p_ap = np.array([x_ap, y_ap, z_ap])

        d_array[i, j] = dist_to_mid(p_o, p_d, p_ap) # ijはlink i+1, ビーコンj+1の距離

###### 混雑度dens，maxRSSI，meanRSSI，検出回数を各ビーコン各timestepごとに入れた配列（ビーコン数23*タイムステップ数） ######
dens_jt = np.zeros((23, 45), dtype = int)
max_jt = np.zeros((23, 45), dtype = int)
mean_jt = np.zeros((23, 45), dtype = int)
#count_jt = np.zeros((23, 45), dtype = int)

grouped = df_data.groupby(['ID', 'timestep']) # listにして渡す必要がある
df_list = [group.reset_index(drop=True) for name, group in grouped]

for group in df_list:
    beacon_id = int(group.loc[0, 'ID'])
    timestep = int(group.loc[0, 'timestep'])

    # ユニークMACアドレス数，最大&平均RSSI
    unique_mac_count = group['MAC'].nunique()
    max_rssi = group['RSSI'].max()
    mean_rssi = group['RSSI'].mean()
    # countはそのタイムステップでの全検出数つまりgroupの長さそのもの
    #count = len(group)

    # numpy配列の要素はシーケンスダ
    dens_jt[beacon_id-1, timestep-1] = unique_mac_count
    max_jt[beacon_id-1, timestep-1] = max_rssi
    mean_jt[beacon_id-1, timestep-1] = mean_rssi
    #count_jt[beacon_id-1, timestep-1] = count

####### リンク接続行列 ####### 
def Imaking(link_data): 
    n = len(link_data)
    I = np.eye(n) # 滞在リンク（吸収リンクにもつけて良い）
    for i in range(n):
        O = link_data.loc[i, 'o'] # 当該linkのOnode
        D = link_data.loc[i, 'd'] # 当該linkのDnode
        for j in range(n): # リンクiのOorDをOないしDとしているリンクjとは接続
            if ((link_data.loc[j, 'o'] == O) or (link_data.loc[j, 'o'] == D)) or (link_data.loc[j, 'd'] == O) or (link_data.loc[j, 'd'] == D): 
                I[i, j] = 1
    return(I)

I = Imaking(df_link)

# 階段リンクについて別途処理 i = 26~44
for i in range(26, 45, 2):
    # 階段には滞在リンク入れない．向きは考えないので面倒なことは考えない
    I[i-1, i-1] = 0
    I[i, i] = 0

# 吸収リンクからは吸収リンクにしか接続しない
for i in range(46, 49):
    I[i-1, :] = 0 # 他には繋がらない
    I[i-1, i-1] = 1 # 自分自身にはつながる 

###### 改札の吸収リンク ######
ddata = [46, 47, 48]
D = len(ddata)

####### パラメタ&タイムステップ設定 #######
xdim = 3
x = np.zeros(xdim)
x[0] = -1
x[-1] = 0.8
sigma = 15 #### 想像できなすぎる．理想は，リンクごとに設定して推定
T = 10 # 10stepで最終リンクには辿り着く人たち，吸収に行くのに+1timestep要すると考えるとT=11が妥当か．

####### 観測モデル ######
####### 観測方程式 ######
def qq(dist):
    #return 
    return (dist/((sigma)**2) + np.exp(-((dist)**2)/2/((sigma)**2)))

##### 重み付き最小二乗法 #####
def weighted_least_squares(coordinates, observations):
    # 各観測点の座標と重みを取り出す
    observed_coordinates = observations[:, :-1]  # 最後の列（重み）を除いた座標
    weights = observations[:, -1]  # 重み
    
    errors = coordinates - observed_coordinates
    weighted_errors = weights * np.sum(errors ** 2, axis=1)
    
    return np.sum(weighted_errors)

##### 観測モデル #####
saikyo_key = [1, 2, 3, 4, 5, 6]
yamate_key = [7, 8, 9, 10, 11, 12, 13]
discriminate_key = [19, 22, 23]
abs_beacons = [16, 20, 21]

def measuring_model(x):
    print(f'パラメタ{x}で観測モデル回す')
    measured_loc = np.zeros(6) # 'userid', 'timestep', 'x', 'y', 'z' 

    for file_name in file_list:
        file_path = os.path.join(search_folder, file_name)
        dfi = pd.read_csv(file_path)
        
        base_name = file_name.split('.')[0]   
        userid = int(base_name)

        nts = dfi['timestep'].nunique()                
        grouped = dfi.groupby('timestep')
        dfi_list = [group.reset_index(drop=True) for name, group in grouped]  

        i_res = np.zeros(6)
            
        for t in range(nts): 
            dfit = dfi_list[t]
            ts = int(dfit.loc[0, 'timestep'])

            result_beacon = dfit.groupby('ID')
            result_beacon_list = [group.reset_index(drop=True) for name, group in result_beacon]

            tdict = {}

            for j in range(len(result_beacon_list)): # 各ビーコンの結果を束ねたdataframe
                dfitj = result_beacon_list[j]
                jscore = 0 
                jid = dfitj.loc[0, 'ID'] # このビーコンのid
                max_rssi = max_jt[jid-1, ts-1] # tsでのbeaconのmaxのrssi

                for k in range(len(dfitj)): 
                    rssi = dfitj.loc[k, 'RSSI']                        
                    jscore += max_rssi / rssi 
                    tdict[jid] = jscore  
            
            # dictにビーコンのidとRSSI実績値による重みが入った．最大ビーコンのidをメモ
            jmax = max(tdict, key = tdict.get)
            
            # t=が最後の時は次の最大ビーコンの座標を参照できないので場合わけ
            if t < nts-1:
                if len(tdict) > 1: # 2点以上で観測されてたら，最小二乗法で一点の推定位置を出すことが可能  
                    observation_array = np.zeros(4)
                    for b in range(len(tdict)):
                        beacon_i = list(tdict.keys())[b] # beacon_iはビーコンのidなので-1してindexにする
                        beacon_i = int(beacon_i)
                        loc_i = beacon_loc_array[beacon_i-1] # 位置座標取得
                        beacon_w = list(tdict.values())[b]

                        result_array = np.insert(loc_i, len(loc_i), beacon_w)
                        observation_array = np.vstack((observation_array, result_array))
                    
                    # 先頭の0の行を削除
                    observation_array = np.delete(observation_array, 0, axis = 0)

                    # 初期座標の設定（仮の値を使用，ある程度それっぽい値を入れないと変な値を返してくるので注意）
                    initial_coordinates = np.array([-11920, -37880, 20])

                    # 重み付き最小二乗法を最小化して推定座標を求める
                    result = minimize(weighted_least_squares, initial_coordinates, args=(observation_array,), method='Nelder-Mead')

                    # 推定された座標を取得
                    estimated_coordinates = result.x
                    estimated_coordinates_with_j = np.insert(estimated_coordinates, 0, jmax)
                    estimated_coordinates_with_ts = np.insert(estimated_coordinates_with_j, 0, ts) # tsを列の先頭に追加

                elif len(tdict) == 1: # その時点tが最後(nts-1)ではなく，かつ観測が1ビーコンだったら，次の観測を参照
                    observation_array = np.zeros(4)

                    beacon_i = list(tdict.keys())[0] # beacon_iはビーコンのidなので-1してindexにする
                    beacon_i = int(beacon_i)
                    loc_i = beacon_loc_array[beacon_i-1] # 位置座標取得
                    beacon_w = list(tdict.values())[0]

                    result_array = np.append(loc_i, float(beacon_w))
                    observation_array = np.vstack((observation_array, result_array))

                    # 次のmaxbeaconを参照
                    dfitt = dfi_list[t+1]
                
                    next_result_beacon = dfitt.groupby('ID')
                    next_result_beacon_list = [group.reset_index(drop=True) for name, group in next_result_beacon]

                    # 時刻t+1の各ビーコンのscoreを記録するdictの準備
                    ttdict = {}
                    for j in range(len(next_result_beacon_list)): # 各ビーコンの結果を束ねたdataframe
                        dfittj = next_result_beacon_list[j]
                        next_jscore = 0 
                        next_jid = dfittj.loc[0, 'ID'] 
                        next_max_rssi = max_jt[next_jid-1, (ts+1)-1] # tsでのbeaconのmaxのrssi

                        for k in range(len(dfittj)): # freq = count / count_tot
                            next_rssi = dfittj.loc[k, 'RSSI']                        
                            next_jscore += next_max_rssi / next_rssi # beacon_eval(rssi, max_rssi) # , freq)
                            ttdict[next_jid] = next_jscore

                    # 次の最大beaconのkeyとscoreを取得して
                    next_max_key = max(ttdict, key = ttdict.get)
                    next_max_score = ttdict[next_max_key]

                    next_max_beacon_loc = beacon_loc_array[next_max_key-1] # 次の最大ビーコンの座標

                    next_result_array = np.insert(next_max_beacon_loc, len(next_max_beacon_loc), next_max_score)
                    observation_array = np.vstack((observation_array, next_result_array))

                    # 先頭の0の行を削除
                    observation_array = np.delete(observation_array, 0, axis = 0)

                    # 初期座標の設定
                    initial_coordinates = np.array([-11920, -37880, 20])
                    result = minimize(weighted_least_squares, initial_coordinates, args=(observation_array,), method='Nelder-Mead')

                    # 推定された座標を取得
                    estimated_coordinates = result.x
                    estimated_coordinates_with_j = np.insert(estimated_coordinates, 0, jmax)
                    estimated_coordinates_with_ts = np.insert(estimated_coordinates_with_j, 0, ts) # tsを列の先頭に追加(ts, x, y, z)
            
            elif t == nts-1:
                last_beacon = max(tdict, key = tdict.get)
                last_beacon = int(last_beacon)
                estimated_coordinates = beacon_loc_array[last_beacon-1] # 最後は一番検出が顕著だったビーコンの座標で
                
                estimated_coordinates_with_j = np.insert(estimated_coordinates, 0, jmax)
                estimated_coordinates_with_ts = np.insert(estimated_coordinates_with_j, 0, ts)

            estimated_coordinates_with_ts_i = np.insert(estimated_coordinates_with_ts, 0, userid) # useridを列の先頭に追加
            i_res = np.vstack((i_res, estimated_coordinates_with_ts_i))

        measured_loc = np.vstack((measured_loc, i_res))
        measured_loc_df = pd.DataFrame(measured_loc, columns = ['userid', 'timestep', 'maxbeacon', 'x', 'y', 'z'])

    return measured_loc_df

x1 = np.array([1, 3])
plot_result = measuring_model(x1)
plot_result.to_csv('/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/split_files/group3_1_2plot.csv')

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_19395/1214740846.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment


パラメタ[1 3]で観測モデル回す


In [59]:
###### 各linkのoとdの座標を入れておく配列 ###### 
link_loc_array = [] 
for i in range(len(df_link)):
    linkid = df_link.loc[i, 'linkid']
    O = df_link.loc[i, 'o']
    D = df_link.loc[i, 'd']

    x_o = df_node[df_node['nodeid'] == O]['x'].iloc[0]
    y_o = df_node[df_node['nodeid'] == O]['y'].iloc[0]
    z_o = df_node[df_node['nodeid'] == O]['floor'].iloc[0] 
    o_loc = np.array([x_o, y_o, z_o])

    x_d = df_node[df_node['nodeid'] == D]['x'].iloc[0]
    y_d = df_node[df_node['nodeid'] == D]['y'].iloc[0]
    z_d = df_node[df_node['nodeid'] == D]['floor'].iloc[0]
    d_loc = np.array([x_d, y_d, z_d])

    loc_tuple = (o_loc, d_loc)

    link_loc_array.append(loc_tuple) # link_loc_array[linkid//2][0]がlinkidのonode, [1]がdnode，[2]が中点，，，，

###### 各ビーコンの座標を入れておく配列 ###### 
beacon_loc_array = []
for j in range(len(df_ble)):
    beacon_id = df_ble.loc[j, 'ID']
    x_j = df_ble[df_ble['ID'] == beacon_id]['x'].iloc[0]
    y_j = df_ble[df_ble['ID'] == beacon_id]['y'].iloc[0]
    z_j = df_ble[df_ble['ID'] == beacon_id]['floor'].iloc[0]
    beacon_loc = np.array([x_j, y_j, z_j])

    beacon_loc_array.append(beacon_loc) # beacon_loc_array[i]がビーコンi+1の位置座標

###### ビーコンとリンク線分の中点間距離を返す関数 ######
def dist_to_mid(p1, p2, x):
    p1 = np.array(p1)
    p2 = np.array(p2)
    x = np.array(x)
    
    segment = p2 - p1
    v1 = x - p1
    v2 = x - p2
    mid_point = (p1 + p2) / 2

    # 垂直ベクトルを計算
    v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
    distance = np.linalg.norm(x - mid_point)
    
    return distance

###### d_arrayの用意 ###### # 各リンク(の中点）とbleビーコンとの距離配列
d_array = np.zeros((len(df_link), len(df_ble)))

for i in range(len(df_link)):
    p_o = link_loc_array[i][0] # o座標 # i=1の時
    p_d = link_loc_array[i][1] # d座標

    for j in range(len(df_ble)):
        x_ap = df_ble.loc[j, 'x']
        y_ap = df_ble.loc[j, 'y']
        z_ap = df_ble.loc[j, 'floor']
        p_ap = np.array([x_ap, y_ap, z_ap])

        d_array[i, j] = dist_to_mid(p_o, p_d, p_ap) # ijはlink i+1, ビーコンj+1の距離

###### 混雑度dens，maxRSSI，meanRSSI，検出回数を各ビーコン各timestepごとに入れた配列（ビーコン数23*タイムステップ数） ######
dens_jt = np.zeros((23, 45), dtype = int)
max_jt = np.zeros((23, 45), dtype = int)
mean_jt = np.zeros((23, 45), dtype = int)
#count_jt = np.zeros((23, 45), dtype = int)

grouped = df_data.groupby(['ID', 'timestep']) # listにして渡す必要がある
df_list = [group.reset_index(drop=True) for name, group in grouped]

for group in df_list:
    beacon_id = int(group.loc[0, 'ID'])
    timestep = int(group.loc[0, 'timestep'])

    # ユニークMACアドレス数，最大&平均RSSI
    unique_mac_count = group['MAC'].nunique()
    max_rssi = group['RSSI'].max()
    mean_rssi = group['RSSI'].mean()
    # countはそのタイムステップでの全検出数つまりgroupの長さそのもの
    #count = len(group)

    # numpy配列の要素はシーケンスダ
    dens_jt[beacon_id-1, timestep-1] = unique_mac_count
    max_jt[beacon_id-1, timestep-1] = max_rssi
    mean_jt[beacon_id-1, timestep-1] = mean_rssi
    #count_jt[beacon_id-1, timestep-1] = count

####### リンク接続行列 ####### 
def Imaking(link_data): 
    n = len(link_data)
    I = np.eye(n) # 滞在リンク（吸収リンクにもつけて良い）
    for i in range(n):
        O = link_data.loc[i, 'o'] # 当該linkのOnode
        D = link_data.loc[i, 'd'] # 当該linkのDnode
        for j in range(n): # リンクiのOorDをOないしDとしているリンクjとは接続
            if ((link_data.loc[j, 'o'] == O) or (link_data.loc[j, 'o'] == D)) or (link_data.loc[j, 'd'] == O) or (link_data.loc[j, 'd'] == D): 
                I[i, j] = 1
    return(I)

I = Imaking(df_link)

# 階段リンクについて別途処理 i = 26~44
for i in range(26, 45, 2):
    # 階段には滞在リンク入れない．向きは考えないので面倒なことは考えない
    I[i-1, i-1] = 0
    I[i, i] = 0

# 吸収リンクからは吸収リンクにしか接続しない
for i in range(46, 49):
    I[i-1, :] = 0 # 他には繋がらない
    I[i-1, i-1] = 1 # 自分自身にはつながる 

###### 改札の吸収リンク ######
ddata = [46, 47, 48]
D = len(ddata)

####### パラメタ&タイムステップ設定 #######
xdim = 3
x = np.zeros(xdim)
x[0] = -1
x[-1] = 0.8
sigma = 15 #### 想像できなすぎる．理想は，リンクごとに設定して推定
T = 10 # 10stepで最終リンクには辿り着く人たち，吸収に行くのに+1timestep要すると考えるとT=11が妥当か．

####### 観測モデル ######
####### 観測方程式 ######
def qq(dist):
    #return 
    return (dist/((sigma)**2) + np.exp(-((dist)**2)/2/((sigma)**2)))

##### 重み付き最小二乗法 #####
def weighted_least_squares(coordinates, observations):
    # 各観測点の座標と重みを取り出す
    observed_coordinates = observations[:, :-1]  # 最後の列（重み）を除いた座標
    weights = observations[:, -1]  # 重み
    
    errors = coordinates - observed_coordinates
    weighted_errors = weights * np.sum(errors ** 2, axis=1)
    
    return np.sum(weighted_errors)

##### 観測モデル #####
saikyo_key = [1, 2, 3, 4, 5, 6]
yamate_key = [7, 8, 9, 10, 11, 12, 13]
discriminate_key = [19, 22, 23]
abs_beacons = [16, 20, 21]

def measuring_model(x):
    print(f'パラメタ{x}で観測モデル回す')
    measured_loc = np.zeros(6) # 'userid', 'timestep', 'x', 'y', 'z' 

    for file_name in file_list:
        file_path = os.path.join(search_folder, file_name)
        dfi = pd.read_csv(file_path)
        
        base_name = file_name.split('.')[0]   
        userid = int(base_name)

        nts = dfi['timestep'].nunique()                
        grouped = dfi.groupby('timestep')
        dfi_list = [group.reset_index(drop=True) for name, group in grouped]  

        i_res = np.zeros(6)
            
        for t in range(nts): 
            dfit = dfi_list[t]
            ts = int(dfit.loc[0, 'timestep'])

            result_beacon = dfit.groupby('ID')
            result_beacon_list = [group.reset_index(drop=True) for name, group in result_beacon]

            tdict = {}

            for j in range(len(result_beacon_list)): # 各ビーコンの結果を束ねたdataframe
                dfitj = result_beacon_list[j]
                jscore = 0 
                jid = dfitj.loc[0, 'ID'] # このビーコンのid
                max_rssi = max_jt[jid-1, ts-1] # tsでのbeaconのmaxのrssi

                for k in range(len(dfitj)): 
                    rssi = dfitj.loc[k, 'RSSI']                        
                    jscore += max_rssi / rssi 
                    tdict[jid] = jscore  
            
            # dictにビーコンのidとRSSI実績値による重みが入った．最大ビーコンのidをメモ
            jmax = max(tdict, key = tdict.get)
            
            # t=が最後の時は次の最大ビーコンの座標を参照できないので場合わけ
            if t < nts-1:
                if len(tdict) > 1: # 2点以上で観測されてたら，最小二乗法で一点の推定位置を出すことが可能  
                    observation_array = np.zeros(4)
                    for b in range(len(tdict)):
                        beacon_i = list(tdict.keys())[b] # beacon_iはビーコンのidなので-1してindexにする
                        beacon_i = int(beacon_i)
                        loc_i = beacon_loc_array[beacon_i-1] # 位置座標取得
                        beacon_w = list(tdict.values())[b]

                        result_array = np.insert(loc_i, len(loc_i), beacon_w)
                        observation_array = np.vstack((observation_array, result_array))
                    
                    # 先頭の0の行を削除
                    observation_array = np.delete(observation_array, 0, axis = 0)

                    # 初期座標の設定（仮の値を使用，ある程度それっぽい値を入れないと変な値を返してくるので注意）
                    initial_coordinates = np.array([-11920, -37880, 20])

                    # 重み付き最小二乗法を最小化して推定座標を求める
                    result = minimize(weighted_least_squares, initial_coordinates, args=(observation_array,), method='Nelder-Mead')

                    # 推定された座標を取得
                    estimated_coordinates = result.x
                    estimated_coordinates_with_j = np.insert(estimated_coordinates, 0, jmax)
                    estimated_coordinates_with_ts = np.insert(estimated_coordinates_with_j, 0, ts) # tsを列の先頭に追加

                elif len(tdict) == 1: # その時点tが最後(nts-1)ではなく，かつ観測が1ビーコンだったら，次の観測を参照
                    observation_array = np.zeros(4)

                    beacon_i = list(tdict.keys())[0] # beacon_iはビーコンのidなので-1してindexにする
                    beacon_i = int(beacon_i)
                    loc_i = beacon_loc_array[beacon_i-1] # 位置座標取得
                    beacon_w = list(tdict.values())[0]

                    result_array = np.append(loc_i, float(beacon_w))
                    observation_array = np.vstack((observation_array, result_array))

                    # 次のmaxbeaconを参照
                    dfitt = dfi_list[t+1]
                
                    next_result_beacon = dfitt.groupby('ID')
                    next_result_beacon_list = [group.reset_index(drop=True) for name, group in next_result_beacon]

                    # 時刻t+1の各ビーコンのscoreを記録するdictの準備
                    ttdict = {}
                    for j in range(len(next_result_beacon_list)): # 各ビーコンの結果を束ねたdataframe
                        dfittj = next_result_beacon_list[j]
                        next_jscore = 0 
                        next_jid = dfittj.loc[0, 'ID'] 
                        next_max_rssi = max_jt[next_jid-1, (ts+1)-1] # tsでのbeaconのmaxのrssi

                        for k in range(len(dfittj)): # freq = count / count_tot
                            next_rssi = dfittj.loc[k, 'RSSI']                        
                            next_jscore += next_max_rssi / next_rssi # beacon_eval(rssi, max_rssi) # , freq)
                            ttdict[next_jid] = next_jscore

                    # 次の最大beaconのkeyとscoreを取得して
                    next_max_key = max(ttdict, key = ttdict.get)
                    next_max_score = ttdict[next_max_key]

                    next_max_beacon_loc = beacon_loc_array[next_max_key-1] # 次の最大ビーコンの座標

                    next_result_array = np.insert(next_max_beacon_loc, len(next_max_beacon_loc), next_max_score)
                    observation_array = np.vstack((observation_array, next_result_array))

                    # 先頭の0の行を削除
                    observation_array = np.delete(observation_array, 0, axis = 0)

                    # 初期座標の設定
                    initial_coordinates = np.array([-11920, -37880, 20])
                    result = minimize(weighted_least_squares, initial_coordinates, args=(observation_array,), method='Nelder-Mead')

                    # 推定された座標を取得
                    estimated_coordinates = result.x
                    estimated_coordinates_with_j = np.insert(estimated_coordinates, 0, jmax)
                    estimated_coordinates_with_ts = np.insert(estimated_coordinates_with_j, 0, ts) # tsを列の先頭に追加(ts, x, y, z)
            
            elif t == nts-1:
                last_beacon = max(tdict, key = tdict.get)
                last_beacon = int(last_beacon)
                estimated_coordinates = beacon_loc_array[last_beacon-1] # 最後は一番検出が顕著だったビーコンの座標で
                
                estimated_coordinates_with_j = np.insert(estimated_coordinates, 0, jmax)
                estimated_coordinates_with_ts = np.insert(estimated_coordinates_with_j, 0, ts)

            estimated_coordinates_with_ts_i = np.insert(estimated_coordinates_with_ts, 0, userid) # useridを列の先頭に追加
            i_res = np.vstack((i_res, estimated_coordinates_with_ts_i))

        measured_loc = np.vstack((measured_loc, i_res))
        measured_loc_df = pd.DataFrame(measured_loc, columns = ['userid', 'timestep', 'maxbeacon', 'x', 'y', 'z'])

    return measured_loc_df


###### 観測モデル2 ######（ODの決定→odlistを作る＆個人の観測モデルに対してのプリズムと，個人の経路選択用のプリズムを用意する


##########################
###### 経路選択モデル ######
##########################

###### 経路選択モデリングのための時空間プリズム ######
### 個人の観測モデルのためのプリズムは別で用意すべしなはず ###
#Ilist = []
#for i in range(L): # 今回はいらないよね．すでにIの設定の時点で与えてあるので
    #I[i, i] = 1 # Iは普通の空間接続行列．これに滞在リンクを追加している→階段自体にはないはず，その他の滞在リンクはすでにIの設定で実施済み
    
# 起点Oからの到達可能性指示変数ioの定義 kから出発して時刻（遷移回数）t以内に到達できるリンクaがII[t, k, a]=1になっている
# 個人の観測に関してはこれを個別に与えないといけない
Itlist = np.zeros((T+1, L, L))
II = np.copy(I)
Itlist[0, :, :] = np.eye(L) # ItlistにはT+1成分あるが，その第一成分(t=0)は単位行列ということ．t=1以降はfor内で計算

for ts in range(1, T+1): 
    Itlist[ts, :, :] = II # 時刻1では普通の接続行列
    II = np.dot(II, I) # 接続条件の積．つまり2回先に接続していたら1，そうでなければ0
    II = np.where(II > 0, 1, 0) 

# 終点Dからの到達可能性指示変数idの定義
Ittlist = np.zeros((T+1, L, L))
for ts in range(1, T+2): # Ittlist[T, ]までしか考えない
    Ittlist[ts-1, :, :] = np.transpose(Itlist[(T+1) - ts, :, :]) 

###### 即時効用行列 ######
def Mset(x): 
    cost = np.exp(df_link['length']/10 * x[0]  + df_link['staire'] * x[1]) # + df_link_integrated['staire_with_esc'] * x[3])
    cost = pd.concat([cost]*L, axis=1)
    cost = cost.T
    cost_numpy = cost.values # DataFrameをNumPy配列に変換
    return cost_numpy

###### 対数尤度関数 ######
def loglikelihood(x):
    beta = x[-1]
    LL = 0
    
    # 観測モデル回し結果をuseridで分割
    measured_loc_df = measuring_model(x)

    grouped = measured_loc_df.groupby('userid')
    df_list = [group for name, group in grouped]
    df_list = [data.reset_index(drop=True) for data in df_list]    
    
    #### 以降個人ごとに処理（OD決定→個人プリズムの作成&経路選択モデル用のプリズムの作成→マッチング→経路データの作成→尤度関数の計算
    for i in range(len(df_list)):
        dfi = df_list[i]
        abs = dfi.loc[len(dfi)-1, 'maxbeacon']
        userid = dfi.loc[0, 'userid']
        maxbeacon_list = dfi['maxbeacon'].tolist()
        Ti = len(dfi) # 個人の観測限界

        # 吸収がハチ公・中央以外だったら飛ばす
        if abs not in abs_beacons:
            continue
        
        # O決める
        first_beacon = int(dfi.loc[0, 'maxbeacon'])
        first_dist_list = d_array[:, first_beacon-1] 
        first_dist_dict = {index + 1: value for index, value in enumerate(first_dist_list)} # keyはlinkidになっている．max_beaconと各リンクの距離が入ってる

        if any(item in discriminate_key for item in maxbeacon_list): # 埼京オンリーにする
            filtered_dict = {key: first_dist_dict[key] for key in first_dist_dict if key in saikyo_key}
            max_key = min(filtered_dict, key = filtered_dict.get) # これが初期リンクとなる．埼京ホームのうち，最もmax_beaconに近いリンク
        else: 
            filtered_dict = {key: first_dist_dict[key] for key in first_dist_dict if key in yamate_key}
            max_key = min(filtered_dict, key = filtered_dict.get)
                  
        a0 = int(max_key) 

        # 次にDを確定
        last_beacon = int(dfi.loc[Ti-1, 'maxbeacon']) # 一番下の行を指定
        last_link = 0
        if last_beacon == 16: # node39
            last_link = 25 
            abs_link = 46
        elif last_beacon == 20: # node28
            last_link = 17
            abs_link = 47
        elif last_beacon == 21: # node29
            last_link = 18   
            abs_link = 48   
        else:
            continue  
        
        # ODの確定
        ao = a0 - 1 
        ad = abs_link -1 # last_link - 1

        ###### OD決まったら時空間プリズム制約Idを入れる #####
        ###### 観測モデル用プリズムIdmと選択モデル用プリズムIdrlを作る ######
        Idrl = np.zeros((T, L, L)) # 
        Idm = np.zeros((Ti, L, L)) # t=Ti-2でlast linkに到達するので．t=Ti-1でabs linkに到達するようにする

        for ts in range(T):
            if ts == 0:
                Idrl[ts, ao, :] = I[ao, :] # →全部利用可能なのおかしっくね，と思うが．
                continue

            #if ts >= 1 and ts <= (len(dfi)):
                # ts>1の各時刻に対して
                # alistはOD（個人）に対して，時刻ts+1において
            alist = np.where(Itlist[ts + 1, ao, :] == Ittlist[ts + 1, ad, :])[0]  # aoからts+1で遷移可能なリンクと，adからts+1までに巻き戻し可能なリンクが共通するところ
                #### Itlist[ts + 1, ao, :]はaoをoとしたときに時刻ts+1に到達可能なリンクa，Ittlist[ts + 1, ad, :]はadをdとするときに時刻ts+1に滞在して良いリンク．これが共に1であるようなリンクをプリズム内に入れる（1にする），と言うのが本来の思想のはず．
            for a in alist:
                if Itlist[ts + 1, ao, a] == Ittlist[ts + 1, ad, a]: # これelseなわけなくね，と思うが．
                    klist = np.where(I[:, a] == 1)[0] # このようなaに接続しているk

                    for k in klist:
                        if len(np.where(Idrl[ts - 1, :, k] == 1)[0]) != 0:
                            Idrl[ts, k, a] = 1 # ある個人が時刻tにkにいた時に利用可能なリンクaが1になっているはず．

            #else: # ts >= len(dfi) and ts <= T
            #    Id[ts, :, :] = np.zeros((L, L))
            #    Id[ts, -3:, -3:] = 1 # 吸収リンクでの滞在のみ可能とする
        
        #### 個人の観測用プリズムIdmを作る前にItmlist，Ittmlistを作り直す必要
        # 個人観測に関しては，時間制約が観測結果から求められているので，これを個別に与えないといけない
        Itmlist = np.zeros((Ti+1, L, L))
        IIm = np.copy(I)
        Itmlist[0, :, :] = np.eye(L) # ItlistにはT+1成分あるが，その第一成分(t=0)は単位行列ということ．t=1以降はfor内で計算

        for tt in range(1, Ti+1): 
            Itmlist[tt, :, :] = IIm # 時刻1では普通の接続行列
            IIm = np.dot(IIm, I) # 接続条件の積．つまり2回先に接続していたら1，そうでなければ0
            IIm = np.where(IIm > 0, 1, 0) 

        # 終点Dからの到達可能性指示変数idの定義
        Ittmlist = np.zeros((Ti+1, L, L))
        for tt in range(1, Ti+2): # Ittlist[T, ]までしか考えない
            Ittmlist[tt-1, :, :] = np.transpose(Itmlist[(Ti+1) - tt, :, :]) 

        for tt in range(Ti):
            # 観測用のプリズム作る
            if tt == 0:
                Idm[tt, ao, :] = I[ao, :] # →全部利用可能なのおかしっくね，と思うが．
                continue

            #if ts >= 1 and ts <= (len(dfi)):
                # ts>1の各時刻に対して
                # alistはOD（個人）に対して，時刻ts+1において
            alist = np.where(Itmlist[tt + 1, ao, :] == Ittmlist[tt + 1, ad, :])[0]  # aoからts+1で遷移可能なリンクと，adからts+1までに巻き戻し可能なリンクが共通するところ
                #### Itlist[ts + 1, ao, :]はaoをoとしたときに時刻ts+1に到達可能なリンクa，Ittlist[ts + 1, ad, :]はadをdとするときに時刻ts+1に滞在して良いリンク．これが共に1であるようなリンクをプリズム内に入れる（1にする），と言うのが本来の思想のはず．
            for a in alist:
                if Itmlist[tt + 1, ao, a] == Ittmlist[tt + 1, ad, a]:
                    klist = np.where(I[:, a] == 1)[0] # このようなaに接続しているk

                    for k in klist:
                        if len(np.where(Idm[tt - 1, :, k] == 1)[0]) != 0:
                            Idm[tt, k, a] = 1 # ある個人が時刻tにkにいた時に利用可能なリンクaが1になっている

        ##### 一応上記までで観測プリズムIdmは作られている
        ##############
        
        ##### 即時効用 #####（これはrlのやつなのでIdrlを使う）
        M = np.zeros((T, L, L))
        for ts in range(T):
            Mts = Idrl[ts, :, :] * Mset(x)
            M[ts, :, :] = Mts

        ##### 価値関数 ##### 
        z = np.ones((T+1, L))
        for t in range(T, 0, -1):
            zii = M[t-1, :, :] * (z[t, :] ** beta) # Mt-1にプリズム条件が入り込んでいる
            zi = zii.sum(axis = 1)
            z[t-1, :] = (zi==0)*1 + (zi!=0)*zi
        
        #print(z)

        dfi_data = pd.DataFrame({'t': range(1, T + 1)})
        # 'k', 'a' 列を追加し、初期化する
        dfi_data['k'] = 0
        dfi_data['a'] = 0
                    
        atk = a0 # 初期値

        ###### loglikelihood内で観測結果ao, a1, .., aT, aabsの列をdfi_dataとして生成する ######
        for t in range(T): # Ti = len(dfi)を超えたら，吸収状態の行を追加していく．列はidとt, k, a
            
            ### 各時刻tにおいて，接続情報Idt[t]と観測結果dfi[t+1]を見て，その時の遷移先(ata=at+1)を決める．それをdfi_data[t, 'a']に追加．次のループでdfi_data[t, 'k']に追加すればいい
            ########### ata = at+1_k である．ここが勘違いの原因だった．
            ## t=0の時はdfi[0]からat=a0を決めていたが，dfi_data
            
            if (t <= (Ti-2)):
                Idmt = Idm[t, :, :] # 時刻tにおいて次に利用可能なリンク（＝aの候補）

                #### t+1にしたら
                #### 期待的には，ここがかなりのそ行列になってて絞られている状況

                # Idmという0または1の値を持つ行列が既に定義されていると仮定します

                # Idmから値が1の要素の位置を取得
                indices = np.where(Idmt == 1)

                # 取得した位置を出力
                print(f"時刻{t}でプリズム制約Idmtの値が1の要素の位置:")
                for i in range(len(indices[0])):
                    print(f"行: {indices[0][i]+1}, 列: {indices[1][i]+1}")
                    ###### 時刻0ではkがaoになっており無条件で接続リンクaが1になっている（コードの処理のまま）

                Idmt1 = Idm[t+1, :, :]
                #print(f'user{userid}の時刻{t}でのプリズムは{Idmt}')        
                dfi_data.loc[t, 'k'] = atk # tの時にいたリンクはatk
                
                a_row = Idmt[atk-1, :]
                ata_cand = np.where(a_row == 1)[0]+1
                # print(f'user{userid}の時刻{t}での接続行列は{Idt}')

                a_row1 = Idmt1[atk-1, :]
                ata_cand2 = np.where(a_row1 == 1)[0]+1

                # 遷移先の位置を特定しようとしているのでt+1にする
                estx = dfi.loc[t+1, 'x']
                esty = dfi.loc[t+1, 'y']
                estz = dfi.loc[t+1, 'z']
                estloc = np.array([estx, esty, estz])

                res_dict = {} 

                if len(ata_cand) == 0:
                    print(f'候補なくなちゃった，oは{ao+1}，Dは{ad+1}，時間制限は{Ti}') # →なくなるはずがない．なぜだ．
                    ### 1以降はプリズムが決まってるが0は効いてない．なので0で間違ったリンク取るとプリズムの外に行ってしまって後続が0になっているぽい
                    print(f'+1したIdは{ata_cand2}') # これも空
                    break

                #print(f'user{userid}の時間制約は{len(dfi)}で時刻{t}での移動先の候補は{ata_cand}，今は{atk}')  #，移動自体は{Ti}までなので{Ti}ではkが{last_link}')

                for ata in ata_cand: # 次リンクataの中点位置(po+pd/2)とt+1でのestimated_locから計算する
                    # print(f'user{userid}の時刻{t}での移動先の候補は{at_cand}')
                    ##### 本当ならここでプリズムが効いてata_candがかなり絞られているはずなのだが
                    p_o = link_loc_array[ata-1][0]
                    p_d = link_loc_array[ata-1][0]
                    dist = dist_to_mid(p_o, p_d, estloc) # np.linalg.norm(est_loc - at_mid_loc)

                    pp = np.exp(np.log(M[t, atk-1, ata-1]) + beta * np.log(z[t+1, ata-1]) - np.log(z[t, atk-1])) # MとかzのサイズはTiよりも大きいので問題なし？

                    # ataの観測尤度
                    q = qq(dist)
                    
                    nume = q ** pp 
                    deno = 0
                        
                    a_prime_row = I[atk-1, :] # prime は，at_1からつながっている全リンク
                    ata_prime_list = np.where(a_prime_row == 1)[0] + 1                        

                    for ata_prime in ata_prime_list:
                        p_o_prime = link_loc_array[ata_prime-1][0]
                        p_d_prime = link_loc_array[ata_prime-1][1]
                        dist_prime = dist_to_mid(p_o_prime, p_d_prime, estloc) # np.linalg.norm(est_loc - at_mid_loc)
                        q_prime = qq(dist_prime)
                        pp = (np.exp(np.log(M[t, atk-1, ata_prime-1]) + beta * np.log(z[t+1, ata_prime-1]) - np.log(z[t, atk-1])))
                        deno += q_prime * pp
                    if deno == 0:
                        print(f'deno=0になちゃった')
                        continue

                    delta = nume / deno 
                    # atにリンク尤度を入れる．これを辞書にする．valueが最大のものを次のat_1にする
                    # res_dict[at] = delta * np.log(pp) # 観測尤度を入れるのでdeltaだけ入る

                    res_dict[ata] = (delta, delta * np.log(pp))

                if not res_dict:
                    print('sss')
                    break

                else:
                    print(f'user{userid}時刻{t}の候補は{res_dict}')
                    max_delta_key = max(res_dict, key=lambda k: res_dict[k][0]) # deltaが最も大きいリンクのリンクidということ
                    dfi_data.loc[t, 'a'] = max_delta_key # これが次のdfi_data.loc[t, 'k']になると言うことじゃ！
                    # at_1 = max_delta_key # これはlinkid．毎回更新される  
                    atk = max_delta_key             # 次のatkなので
                    
                    #print(f'時刻{t}の次は{max_delta_key}へ移動')

                    #LL += res_dict[max_delta_key][1] # 最大リンクのdelta*np.log(pp)

            #elif t == Ti-1: # 最後→吸収の行をくわえる
            #    dfi_data.loc[t, 'k'] = abs_link # last_link
            #    dfi_data.loc[t, 'a'] = abs_link
                #print(f'時刻{t}は{last_link}から{abs_link}')

            else: # t > len(dfi)の時は吸収→吸収だね
                dfi_data.loc[t, 'k'] = abs_link
                dfi_data.loc[t, 'a'] = abs_link                
                #print(f'時刻{t}は{abs_link}から{abs_link}')

        ####### ここまでで分析用にdfiを加工
        print(f'user{userid}のdfi_dataは{dfi_data}')
        print(f'user{userid}のdfiは{dfi}')

        ##### 以上で時刻tにどのリンクを選んだかを確定的に推定し，以下で尤度計算に持ち込む ######
        klink = list(dfi_data.loc[:, 'k'])
        alink = list(dfi_data.loc[:, 'a'])

        for t in range(T): # 吸収のところは実質的には関係ないので飛ばして良いのでは
            k = int(klink[t]) - 1 # at
            a = int(alink[t]) - 1 # at+1
            
            #pp = np.exp(np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k]))
            # print(f'個人{od+1}の時刻{t+1}の遷移は{k+1}から{a+1}で，その即時効用は{M[t, k, a]}，{a+1}の期待効用は{z[t+1, a]}')
            
            ##### 
            ##### 吸収の時はest_locとかの概念ない！！！！]
            """
            # t+1の観測を参照
            est_x = dfi.loc[t+1, 'x']
            est_y = dfi.loc[t+1, 'y']
            est_z = dfi.loc[t+1, 'z']
            est_loc = np.array([est_x, est_y, est_z])

            # q(estimated_x, selected_link) # linkの中点at_mid_locを取得する必要がある
            # 考えるのは遷移先のリンクのリンク尤度なのでa
            p_o = link_loc_array[a][0]
            p_d = link_loc_array[a][1]

            dist = dist_to_mid(p_o, p_d, est_loc) # np.linalg.norm(est_loc - at_mid_loc)

            # 一応観測方程式と呼べるものはこれか．
            q = qq(dist) # = dist/((sigma)**2) + np.exp(-((dist)**2)/2/((sigma)**2)) # レイリー分布

            k_row = I[k, :] # atからつながっているやつ
            a_prime_list = [] # at+1の候補集合
            a_prime_list = np.where(k_row == 1)[0]+1

            for a_prime in a_prime_list:
                p_o_prime = link_loc_array[a_prime-1][0]
                p_d_prime = link_loc_array[a_prime-1][1]
                dist_prime = dist_to_mid(p_o_prime, p_d_prime, est_loc) # np.linalg.norm(est_loc - at_mid_loc)
                q_prime = qq(dist_prime)
                        #dist = d_array[at_prime-1, max_beacon-1]
                        #pmat_prime = mq(dist, min_dist)
                pp = (np.exp(np.log(M[t, k, a_prime-1]) + beta * np.log(z[t+1, a_prime-1]) - np.log(z[t, k])))
                deno += q_prime * pp
            if deno == 0:
                continue
            """
            if M[t, k, a] == 0:
                break

            logp = np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])
            ppp = np.exp(logp)
            #pppp = (ppp==0)*1 + (ppp!=0)*ppp
            LL += np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])
            print(f'LLは{np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])}')
            #delta = nume / deno 
            # 最後のリンクへの遷移が分かれば吸収への遷移とか吸収リンク内の遷移とかは確率考える必要なくね．
            #elif t == len(dfi)-1:
            #    delta = nume / deno
 
            #    LL += delta * logp
            print(f'user{userid}の時刻{t}の{k+1}から{a+1}への遷移,Mは{M[t, k, a]},ztは{z[t+1, a]}, zttは{z[t, k]}pppは{ppp}')

            #else:
            #    delta = nume / deno 
            #    logp = np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])

                #print(f'時刻{t}の{k}から{a}への遷移．deltaは{delta}, Mは{M[t, k, a]},ztは{z[t+1, a]}, zttは{z[t, k]}loppは{logp}')

            #    LL += delta * logp

    return -LL


In [60]:
x = np.array((-2,0,0.8))

print(loglikelihood(x))

パラメタ[-2.   0.   0.8]で観測モデル回す
時刻0でプリズム制約Idmtの値が1の要素の位置:
行: 4, 列: 3
行: 4, 列: 4
行: 4, 列: 5
行: 4, 列: 28
行: 4, 列: 38
user51.0時刻0の候補は{3: (1.955782527762626, -1.7204615911781562), 4: (2.170085843559253, -1.9089798024089342), 5: (2.1396047331472863, -1.8821661976365052), 28: (2.225483261536703, -1.957711769551353), 38: (1.759539454315333, -1.5478306030145277)}
時刻1でプリズム制約Idmtの値が1の要素の位置:
行: 3, 列: 38
行: 4, 列: 38
行: 5, 列: 40
行: 28, 列: 38
行: 38, 列: 39
user51.0時刻1の候補は{38: (1.176670192966376, -0.5984766312070398)}
時刻2でプリズム制約Idmtの値が1の要素の位置:
行: 38, 列: 39
行: 39, 列: 14
行: 39, 列: 41
行: 40, 列: 41
user51.0時刻2の候補は{39: (0.6814373024255619, -0.055717056950025635)}
時刻3でプリズム制約Idmtの値が1の要素の位置:
行: 14, 列: 14
行: 14, 列: 15
行: 39, 列: 14
行: 41, 列: 14
user51.0時刻3の候補は{14: (1.030122497369288, -2.4773711764831603)}
時刻4でプリズム制約Idmtの値が1の要素の位置:
行: 14, 列: 15
行: 15, 列: 15
行: 15, 列: 16
行: 15, 列: 17
行: 15, 列: 43
user51.0時刻4の候補は{15: (1.109216630615098, -4.6368086787631295)}
時刻5でプリズム制約Idmtの値が1の要素の位置:
行: 15, 列: 16
行: 16, 列: 16
行: 16, 

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(

時刻0でプリズム制約Idmtの値が1の要素の位置:
行: 9, 列: 8
行: 9, 列: 9
行: 9, 列: 10
行: 9, 列: 30
行: 9, 列: 36
user158.0時刻0の候補は{8: (2.5443024818868247, -4.65259263665141), 9: (2.091692259444142, -3.824935153627482), 10: (1.2861502078960103, -2.3518952756148095), 30: (2.512092285549422, -4.593692044692957), 36: (2.1610536562849147, -3.9517716152929667)}
時刻1でプリズム制約Idmtの値が1の要素の位置:
候補なくなちゃった，oは9，Dは47，時間制限は6
+1したIdは[]
user158.0のdfi_dataは    t  k  a
0   1  9  8
1   2  8  0
2   3  0  0
3   4  0  0
4   5  0  0
5   6  0  0
6   7  0  0
7   8  0  0
8   9  0  0
9  10  0  0
user158.0のdfiは   userid  timestep  maxbeacon             x             y          z
0   158.0      11.0        3.0 -11928.903640 -37911.192810  20.500035
1   158.0      12.0        4.0 -11940.194643 -37883.943942  20.500029
2   158.0      13.0        5.0 -11946.067864 -37868.587982  20.499968
3   158.0      14.0        6.0 -11952.812548 -37851.673486  19.083327
4   158.0      15.0       20.0 -11958.568978 -37843.368153  17.292005
5   158.0      16.0      

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(

時刻0でプリズム制約Idmtの値が1の要素の位置:
行: 4, 列: 3
行: 4, 列: 4
行: 4, 列: 5
行: 4, 列: 28
行: 4, 列: 38
user224.0時刻0の候補は{3: (1.3954038870175827, -1.2275080474007756), 4: (1.4994304054707857, -1.3190180322391531), 5: (1.2311496867725373, -1.0830170118690456), 28: (1.504163353084149, -1.3231815087332217), 38: (1.46062721231704, -1.2848836627535756)}
時刻1でプリズム制約Idmtの値が1の要素の位置:
行: 3, 列: 38
行: 4, 列: 38
行: 5, 列: 40
行: 28, 列: 38
行: 38, 列: 39
user224.0時刻1の候補は{38: (1.337949702834919, -0.6805064287882269)}
時刻2でプリズム制約Idmtの値が1の要素の位置:
行: 38, 列: 39
行: 39, 列: 14
行: 39, 列: 41
行: 40, 列: 41
user224.0時刻2の候補は{39: (0.7257321602444978, -0.05933878283281224)}
時刻3でプリズム制約Idmtの値が1の要素の位置:
行: 14, 列: 14
行: 14, 列: 15
行: 39, 列: 14
行: 41, 列: 14
user224.0時刻3の候補は{14: (1.050248569747368, -2.5257729459160196)}
時刻4でプリズム制約Idmtの値が1の要素の位置:
行: 14, 列: 15
行: 15, 列: 15
行: 15, 列: 16
行: 15, 列: 17
行: 15, 列: 43
user224.0時刻4の候補は{15: (0.8121701370950959, -3.395078505292943)}
時刻5でプリズム制約Idmtの値が1の要素の位置:
行: 15, 列: 16
行: 16, 列: 16
行: 16, 列: 18
行: 16, 列: 45
行: 1

...

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:540: RuntimeWarning: divide by zero encountered in log
  pp = (np.exp(np.log(M[t, atk-1, ata_prime-1]) + beta * np.log(z[t+1, ata_prime-1]) - np.log(z[t, atk-1])))



user1144.0のdfiは   userid  timestep  maxbeacon             x             y          z
0  1144.0       8.0        5.0 -11949.282714 -37843.147053  17.773786
1  1144.0       9.0        6.0 -11948.259999 -37858.241999  20.500027
2  1144.0      10.0        6.0 -11948.259999 -37858.241999  20.500027
3  1144.0      11.0        6.0 -11958.799844 -37843.034941  17.220115
4  1144.0      12.0       20.0 -11954.288511 -37835.233988  14.799971
5  1144.0      13.0       20.0 -11967.048240 -37821.298702  14.799957
6  1144.0      14.0       20.0 -11966.577000 -37831.814000  14.800000
LLは-3.571481081402359
user1144.0の時刻0の11から10への遷移,Mは0.006737946999085467,ztは8.745855531142329e-08, zttは5.408090563771578e-07pppは0.028114183435176538
LLは-0.2982628286484612
user1144.0の時刻1の10から11への遷移,Mは0.006737946999085467,ztは5.366313122879253e-07, zttは8.745855531142329e-08pppは0.7421062673263309
LLは-0.15457527390713466
user1144.0の時刻2の11から12への遷移,Mは0.006737946999085467,ztは6.201854009671608e-06, zttは5.366313122879253e-07pppは0.8

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:550: RuntimeWarning: divide by zero encountered in log
  res_dict[ata] = (delta, delta * np.log(pp))
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment



LLは-2.2464034785876628
user1152.0の時刻0の12から12への遷移,Mは0.006737946999085467,ztは1.0428906064676416e-06, zttは1.0440450290066954e-06pppは0.10577897761549823
LLは-0.8290583282934634
user1152.0の時刻1の12から42への遷移,Mは0.006737946999085467,ztは6.124485842737304e-06, zttは1.0428906064676416e-06pppは0.4364600949902132
LLは-2.5978699473135354
user1152.0の時刻2の42から13への遷移,Mは0.006737946999085467,ztは6.135879572257194e-06, zttは6.124485842737304e-06pppは0.074431953463074
LLは-0.1798214147035644
user1152.0の時刻3の13から44への遷移,Mは0.006737946999085467,ztは0.0001263476695089678, zttは6.135879572257194e-06pppは0.8354193917098882
LLは-0.0034951755644190285
user1152.0の時刻4の44から45への遷移,Mは0.006737946999085467,ztは0.006908792309863981, zttは0.0001263476695089678pppは0.9965109254515835
LLは-0.019435998615532135
user1152.0の時刻5の45から18への遷移,Mは0.006737946999085467,ztは1.0070290519386393, zttは0.006908792309863981pppは0.9807516626442712
LLは0.0
user1152.0の時刻6の48から48への遷移,Mは1.0,ztは1.0, zttは1.0pppは1.0
LLは0.0
user1152.0の時刻7の48から48への遷移,Mは1.0,ztは1.0, zttは1.0pppは

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment


In [70]:


###### 観測モデル2 ######
# モデル1の結果から経路選択モデルの情報使わずに実施．個人ごとにt, k, aの列(要はdfi_data)を作成する
def measuring_model2(x):

    # モデル1を回す
    measured_loc_df = measuring_model(x)
    grouped = measured_loc_df.groupby('userid')
    df_list = [group for name, group in grouped]
    df_list = [data.reset_index(drop=True) for data in df_list]   

    # アウトプットの準備
    columns = ['userid', 'timestep', 'k', 'a']
    result_df = pd.DataFrame(columns = columns)
    
    # 以降個人ごとに処理（OD決定→個人プリズムの作成&経路選択モデル用のプリズムの作成→マッチング→経路データの作成）
    for i in range(len(df_list)):
        dfi = df_list[i]
        abs = dfi.loc[len(dfi)-1, 'maxbeacon']
        userid = int(dfi.loc[0, 'userid'])
        maxbeacon_list = dfi['maxbeacon'].tolist()
        Ti = len(dfi) # 個人の観測限界

        ###### 個人ごとにdfi_dataを入れるのでよくね ######
        dfi_data = pd.DataFrame(columns = columns)
        #dfi_data['userid'] = userid
        #test = dfi_data.loc[0, 'userid']
        # useridの値をdfi_dataの'userid'列に代入
        dfi_data['userid'] = [userid] * T

        # dfi_dataの行数をTに設定
        dfi_data = dfi_data.head(T)


        print(f'userid{userid}の時user列の値は')

        # 吸収がハチ公・中央以外だったら飛ばす
        if abs not in abs_beacons:
            continue
        
        # O決める
        first_beacon = int(dfi.loc[0, 'maxbeacon'])
        first_dist_list = d_array[:, first_beacon-1] 
        first_dist_dict = {index + 1: value for index, value in enumerate(first_dist_list)} # keyはlinkidになっている．max_beaconと各リンクの距離が入ってる

        if any(item in discriminate_key for item in maxbeacon_list): # 埼京オンリーにする
            filtered_dict = {key: first_dist_dict[key] for key in first_dist_dict if key in saikyo_key}
            max_key = min(filtered_dict, key = filtered_dict.get) # これが初期リンクとなる．埼京ホームのうち，最もmax_beaconに近いリンク
        else: 
            filtered_dict = {key: first_dist_dict[key] for key in first_dist_dict if key in yamate_key}
            max_key = min(filtered_dict, key = filtered_dict.get)
                  
        a0 = int(max_key) 

        # 次にDを確定
        last_beacon = int(dfi.loc[Ti-1, 'maxbeacon']) # 一番下の行を指定
        last_link = 0
        if last_beacon == 16: # node39
            last_link = 25 
            abs_link = 46
        elif last_beacon == 20: # node28
            last_link = 17
            abs_link = 47
        elif last_beacon == 21: # node29
            last_link = 18   
            abs_link = 48   
        else:
            continue
        
        # ODの確定
        ao = a0 - 1 
        ad = abs_link -1 
        atk = a0 # 初期値

        # OD決まったら時空間プリズム制約Idを入れる（観測モデル内ではIdmのみ，Idrlは不要）
        Idm = np.zeros((Ti, L, L)) # t=Ti-2でlast linkに到達するので．t=Ti-1でabs linkに到達するようにする
        Itmlist = np.zeros((Ti+1, L, L))
        IIm = np.copy(I)
        Itmlist[0, :, :] = np.eye(L) 

        for tt in range(1, Ti+1): 
            Itmlist[tt, :, :] = IIm 
            IIm = np.dot(IIm, I) 
            IIm = np.where(IIm > 0, 1, 0) 

        Ittmlist = np.zeros((Ti+1, L, L))
        for tt in range(1, Ti+2): 
            Ittmlist[tt-1, :, :] = np.transpose(Itmlist[(Ti+1) - tt, :, :]) 

        for tt in range(Ti):
            if tt == 0:
                Idm[tt, ao, :] = I[ao, :] # →全部利用可能なのおかしっくね，と思うが．
                continue

            alist = np.where(Itmlist[tt + 1, ao, :] == Ittmlist[tt + 1, ad, :])[0]  
            for a in alist:
                if Itmlist[tt + 1, ao, a] == Ittmlist[tt + 1, ad, a]:
                    klist = np.where(I[:, a] == 1)[0] 

                    for k in klist:
                        if len(np.where(Idm[tt - 1, :, k] == 1)[0]) != 0:
                            Idm[tt, k, a] = 1 

        # 時刻tでのプリズム条件と観測結果から各時刻のk, aを決めていく
        for t in range(T): # 列はidとt, k, a．ata = at+1_k
            if (t <= (Ti-2)):
                Idmt = Idm[t+1, :, :] # 時刻tにおいて次に利用可能なリンク（＝aの候補）#### t+1にしたら上手くいってるけど，，ちょっと謎

                dfi_data.loc[t, 'k'] = atk # tの時にいたリンクはatk
                a_row = Idmt[atk-1, :]
                ata_cand = np.where(a_row == 1)[0]+1

                # 遷移「先」の位置を特定しようとしているのでt+1にする
                estx = dfi.loc[t+1, 'x']
                esty = dfi.loc[t+1, 'y']
                estz = dfi.loc[t+1, 'z']
                estloc = np.array([estx, esty, estz])

                res_dict = {} 

                if len(ata_cand) == 0:
                    #print(f'候補なくなちゃった，oは{ao+1}，Dは{ad+1}，時間制限は{Ti}') # →なくなるはずがない．なぜだ．
                    break

                for ata in ata_cand: # 次リンクataの中点位置(po+pd/2)とt+1でのestimated_locから計算する
                    p_o = link_loc_array[ata-1][0]
                    p_d = link_loc_array[ata-1][0]
                    dist = dist_to_mid(p_o, p_d, estloc) 

                    q = qq(dist) # ataの観測尤度
                    res_dict[ata] = q # 事前情報による補正なし（プリズムという強い事前情報はある）

                if not res_dict:
                    print('sss')
                    break

                else:
                    max_delta_key = max(res_dict, key = res_dict.get)
                    dfi_data.loc[t, 'a'] = max_delta_key    # これが次のdfi_data.loc[t, 'k']になると言うことじゃ！
                    atk = max_delta_key                     # 次のatkなので

            else: # t > len(dfi)の時は吸収→吸収だね
                dfi_data.loc[t, 'k'] = abs_link
                dfi_data.loc[t, 'a'] = abs_link   
            
            dfi_data.loc[t, 'timestep'] = t
        
        if dfi_data.isnull().values.any():
            continue
        else:
            print(f'user{userid}のdfi_dataは{dfi_data}')
            # dfi_dataをresult_dfにくっつけたい（カラム名を揃えて）
            result_df = pd.concat([result_df, dfi_data], axis = 0, ignore_index = True)

    return result_df             

In [71]:
x = np.array((-2,0,0.8))

print(measuring_model2(x))

パラメタ[-2.   0.   0.8]で観測モデル回す
userid0の時user列の値は
userid51の時user列の値は
user51のdfi_dataは   userid timestep   k   a
0      51        0   4  38
1      51        1  38  39
2      51        2  39  14
3      51        3  14  15
4      51        4  15  16
5      51        5  16  18
6      51        6  18  48
7      51        7  48  48
8      51        8  48  48
9      51        9  48  48
userid75の時user列の値は
user75のdfi_dataは   userid timestep   k   a
0      75        0   9  30
1      75        1  30  31
2      75        2  31  24
3      75        3  24  25
4      75        4  25  46
5      75        5  46  46
6      75        6  46  46
7      75        7  46  46
8      75        8  46  46
9      75        9  46  46
userid100の時user列の値は
userid141の時user列の値は
userid147の時user列の値は
userid149の時user列の値は
user149のdfi_dataは   userid timestep   k   a
0     149        0   2   3
1     149        1   3   4
2     149        2   4  28
3     149        3  28  29
4     149        4  29  25
5     149        5  25  46
6  

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(

user327のdfi_dataは   userid timestep   k   a
0     327        0   1   1
1     327        1   1   2
2     327        2   2   2
3     327        3   2   3
4     327        4   3  28
5     327        5  28  29
6     327        6  29  25
7     327        7  25  46
8     327        8  46  46
9     327        9  46  46
userid331の時user列の値は
userid336の時user列の値は
user336のdfi_dataは   userid timestep   k   a
0     336        0   2   2
1     336        1   2  26
2     336        2  26  27
3     336        3  27  25
4     336        4  25  46
5     336        5  46  46
6     336        6  46  46
7     336        7  46  46
8     336        8  46  46
9     336        9  46  46
userid343の時user列の値は
userid359の時user列の値は
user359のdfi_dataは   userid timestep   k   a
0     359        0  12  13
1     359        1  13  44
2     359        2  44  45
3     359        3  45  18
4     359        4  18  48
5     359        5  48  48
6     359        6  48  48
7     359        7  48  48
8     359        8  48  48
9    

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(

userid517の時user列の値は
user517のdfi_dataは   userid timestep   k   a
0     517        0   9  30
1     517        1  30  31
2     517        2  31  24
3     517        3  24  25
4     517        4  25  46
5     517        5  46  46
6     517        6  46  46
7     517        7  46  46
8     517        8  46  46
9     517        9  46  46
userid585の時user列の値は
user585のdfi_dataは   userid timestep   k   a
0     585        0   4   4
1     585        1   4  38
2     585        2  38  39
3     585        3  39  14
4     585        4  14  15
5     585        5  15  16
6     585        6  16  18
7     585        7  18  48
8     585        8  48  48
9     585        9  48  48
userid600の時user列の値は
user600のdfi_dataは   userid timestep   k   a
0     600        0   3   4
1     600        1   4  28
2     600        2  28  29
3     600        3  29  25
4     600        4  25  46
5     600        5  46  46
6     600        6  46  46
7     600        7  46  46
8     600        8  46  46
9     600        9  46  4

...

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(


userid893の時user列の値は
userid901の時user列の値は
user901のdfi_dataは   userid timestep   k   a
0     901        0   9   9
1     901        1   9   9
2     901        2   9  30
3     901        3  30  31
4     901        4  31  24
5     901        5  24  25
6     901        6  25  46
7     901        7  46  46
8     901        8  46  46
9     901        9  46  46
userid912の時user列の値は
userid913の時user列の値は
userid922の時user列の値は
userid928の時user列の値は
user928のdfi_dataは   userid timestep   k   a
0     928        0   9   9
1     928        1   9   9
2     928        2   9  30
3     928        3  30  31
4     928        4  31  24
5     928        5  24  25
6     928        6  25  46
7     928        7  46  46
8     928        8  46  46
9     928        9  46  46
userid938の時user列の値は
user938のdfi_dataは   userid timestep   k   a
0     938        0  12  42
1     938        1  42  43
2     938        2  43  17
3     938        3  17  47
4     938        4  47  47
5     938        5  47  47
6     938        6  47  4

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(

user1014のdfi_dataは   userid timestep   k   a
0    1014        0  11  12
1    1014        1  12  12
2    1014        2  12  13
3    1014        3  13  42
4    1014        4  42  43
5    1014        5  43  17
6    1014        6  17  47
7    1014        7  47  47
8    1014        8  47  47
9    1014        9  47  47
userid1018の時user列の値は
userid1057の時user列の値は
user1057のdfi_dataは   userid timestep   k   a
0    1057        0   3  38
1    1057        1  38  39
2    1057        2  39  14
3    1057        3  14  15
4    1057        4  15  17
5    1057        5  17  47
6    1057        6  47  47
7    1057        7  47  47
8    1057        8  47  47
9    1057        9  47  47
userid1090の時user列の値は
user1090のdfi_dataは   userid timestep   k   a
0    1090        0   3   4
1    1090        1   4  38
2    1090        2  38  39
3    1090        3  39  14
4    1090        4  14  15
5    1090        5  15  17
6    1090        6  17  47
7    1090        7  47  47
8    1090        8  47  47
9    1090        9 

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_1070/3911254180.py:45: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(